In [ ]:
# import os
# os.environ["SSL_CERT_FILE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"
# os.environ["REQUESTS_CA_BUNDLE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"

In [ ]:
from qdrant_client import QdrantClient, models
import requests
from fastembed import TextEmbedding
import json
import random
import pandas as pd
from tqdm.auto import tqdm
import uuid
import os


In [3]:
# import the data 
with open('../data/processed_data/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)


In [4]:
documents[11]

{'city': 'Cairo',
 'section': 'Stay safe',
 'subsection': 'Emergency services',
 'text': 'Ambulance , ☏ 123 .',
 'id': 'bff9ff8c'}

In [5]:
# Ground Truth data
df_gt = pd.read_csv('../data/result/groud-truth-retrieval.csv')

In [6]:
df_gt.head()

,id,city,question
0,f7845786,Cairo,What is the name of the oldest known pyramid i...
1,f7845786,Cairo,Which pyramid in Dahshur has an entrance to th...
2,f7845786,Cairo,What is the distinctive feature of the Bent Py...
3,f7845786,Cairo,How many pyramids are mentioned to be in the D...
4,f7845786,Cairo,What is the atmosphere around Dahshur Pyramids...


In [7]:
ground_truth = df_gt.to_dict(orient='records')

In [8]:
ground_truth[55]

{'id': 'bff9ff8c',
 'city': 'Cairo',
 'question': 'What number should I dial for an ambulance in Cairo?'}

In [9]:
#connecting to local Qdrant instance
qdrant_client = QdrantClient(url="http://localhost:6333")

/tmp/ipykernel_1508/2923977033.py:2: UserWarning: Qdrant client version 1.15.1 is incompatible with server version 1.12.4. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  qdrant_client = QdrantClient(url="http://localhost:6333")


In [10]:
qdrant_client

In [11]:
emb_models = TextEmbedding.list_supported_models()
print(f"There are {len(emb_models)} models supported")
emb_models[0:10]

There are 30 models supported


[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [12]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

## Create collection 

In [89]:
# Define the collection name
collection_name = "traveller-rag"
# Delete a connection 
if qdrant_client.collection_exists(collection_name=collection_name):
    qdrant_client.delete_collection(collection_name=collection_name)
    print(f"Delete the currrent collection {collection_name}")


Delete the currrent collection traveller-rag


In [91]:
# Create the collection with specified vector parameters
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size = 512,
        distance=models.Distance.COSINE
    )
)

True

## Create, Embed & Insert Points into the Collection


In [92]:
points = []
id = 0

for doc in documents:
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['text'], model=model_handle),
        payload={
            "id": doc['id'],
            "text": doc['text'],
            "city": doc['city'],
            "section": doc['section'],
            "subsection": doc['subsection']

        }
    )

    points.append(point)

    id+=1

In [93]:
qdrant_client.upsert(
    collection_name=collection_name, 
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# Start the evaluation process

In [13]:
#Hit Rate (HR) or Recall at k
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [14]:
# Mean Reciprocal Rank (MRR)
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [94]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        gt_id = q['id']
        results = search_function(q)

        points = results.points if hasattr(results, "points") else results

        relevance = [
            point.payload.get('id') == gt_id
            for point in points
            if hasattr(point, "payload")
        ]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


# Semantic Search

## Basline

In [95]:
def qdrant_search(query, limit=5):

    results = qdrant_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        limit=limit,
        with_payload=True
    
    )

    return results

In [96]:
basline_qd_search = evaluate(ground_truth, lambda q: qdrant_search(q['question']))
print(basline_qd_search)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2715/2715 [00:32<00:00, 83.06it/s]

{'hit_rate': 0.8593001841620627, 'mrr': 0.7512829957028841}


return more result improve the evaluation matrix 

## Search with filter


In [97]:
qdrant_client.create_payload_index(
    collection_name=collection_name, 
    field_name="city",
    field_schema="keyword"
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [98]:
def qd_search_filter(query, city, limit):
    results=qdrant_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="city",
                    match=models.MatchValue(value=city)
                )
            ]
        ),
        limit = limit,
        with_payload=True
    )

    return results

In [99]:
filter_qd_search = evaluate(ground_truth, lambda q: qd_search_filter(q['question'], q['city'], 5))
print(filter_qd_search)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2715/2715 [00:31<00:00, 86.39it/s]

{'hit_rate': 0.8674033149171271, 'mrr': 0.7655125844076113}


In [100]:
results = qd_search_filter("What museum I should visit?", "Rome", 5)

points = [
    {
        "score": point.score,
        "payload": point.payload
    }
    for point in results.points
]

print(json.dumps(points, indent=2))

[
  {
    "score": 0.84569573,
    "payload": {
      "id": "1eed9c8a",
      "text": "If you have plenty of time there is absolutely no shortage of other museums covering a wide variety of interests. Examples include the Museum of the Walls (see Rome/South ), the Musical Instrument Museum and a museum devoted to the liberation of Rome from German occupation in the Second World War ( Rome/Esquilino-San Giovanni ).",
      "city": "Rome",
      "section": "See",
      "subsection": "Museums"
    }
  },
  {
    "score": 0.8343532,
    "payload": {
      "id": "18f7911a",
      "text": "If you are in Rome for the Arts there are several world-class museums in the city. The natural starting point is a visit to the area of Villa Borghese in Rome/North Center , where there is a cluster of art museums in and around the Borghese Gardens. Galleria Borghese houses a previously private art collection of the Borghese family, Museo Nazionale di Villa Giulia is home of the world's largest Etruscan ar

# Sparse vector search with BM25

In [16]:
collection_name_sparse = "traveller-sparse"

In [20]:
# Create a collection for sparse vector parameters 
qdrant_client.create_collection(
    collection_name=collection_name_sparse,
    sparse_vectors_config={
        "bm25":models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    }
)

In [21]:
# Send the points to the collection
qdrant_client.upsert(
    collection_name=collection_name_sparse,
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "bm25":models.Document(
                    text=doc["text"],
                    model="Qdrant/bm25",
                )
            },
            payload={
            "id": doc['id'],
            "text": doc['text'],
            "city": doc['city'],
            "section": doc['section'],
            "subsection": doc['subsection']

            }
        )
        for doc in documents
    ]
)

Fetching 18 files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:01<00:00, 14.69it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

## Running sparse vector search with BM25

In [24]:
def bm_search(query: str, limit: int =5) -> list[models.ScoredPoint]:
    results = qdrant_client.query_points(
        collection_name=collection_name_sparse,
        query=models.Document(
            text=query,
            model="Qdrant/bm25",
        ),
        using="bm25",
        limit=limit,
        with_payload=True
    )

    return results.points

In [51]:
bm_qd_search = evaluate(ground_truth, lambda q: bm_search(q['question']))
print(bm_qd_search)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2715/2715 [00:09<00:00, 297.88it/s]

{'hit_rate': 0.8471454880294659, 'mrr': 0.7386863106200116}


In [55]:
results = bm_search("love")
for point in results:
    print(
        f"Score: {point.score}\n"
        f"City: {point.payload.get('city')}\n"
        f"Section: {point.payload.get('section')}\n"
        f"Subsection: {point.payload.get('subsection')}\n"
        f"Text: {point.payload.get('text')}"
    )
    print("=="*50)
    
        


Score: 7.376323
City: Rome
Section: See
Subsection: Rome for kids
Text: The Time Elevator [dead link] . Via dei Santi Apostoli, 20 on a side street between Piazza Venezia and the Trevi Fountain. Daily 10.30 to 19.30. "Five-dimensional" shows on the Origins of Life and on the History of Rome, plus "The House of Horrors". Not for the faint-hearted: your seats move all over the place. Kids love it.
Score: 7.286756
City: Rome
Section: See
Subsection: The Piazzas
Text: Moving back to the Modern Centre you have to see the Trevi Fountain , surely a part of everyone's Roman holiday. Visitors are always amazed that such a big and famous fountain is tucked away in a small piazza in the middle of side streets. Take extra-special care of your possessions here. Further up the Via del Tritone you will come to Piazza Barberini , now full of traffic but the lovely Bernini fountain is not to be missed.
Score: 7.0142126
City: London
Section: Drink
Subsection: Nightclubs
Text: The area around Mayfair is 

# Multi-step search dense vector search, and rerank  with sparse search

In [57]:
collection_name_dense_sparse="traveller-dense-and-sparse"

In [59]:
# Create the collection with both vector types
qdrant_client.create_collection(
    collection_name=collection_name_dense_sparse,
    vectors_config={
        'jina-small':models.VectorParams(
            size=512,
            distance=models.Distance.COSINE
        ),
    },
    sparse_vectors_config={
        "bm25":models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    }
)

In [60]:
qdrant_client.upsert(
    collection_name=collection_name_dense_sparse,
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "jina-small": models.Document(
                    text=doc["text"],
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                "bm25": models.Document(
                    text=doc["text"], 
                    model="Qdrant/bm25",
                ),
            },
            payload={
            "id": doc['id'],
            "text": doc['text'],
            "city": doc['city'],
            "section": doc['section'],
            "subsection": doc['subsection']}

        )
        for doc in documents
    ]
)


Fetching 5 files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.17s/it]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [70]:
def dense_sparse_search(query: str, limit: int = 5) -> list[models.ScoredPoint]:
    results = qdrant_client.query_points(
        collection_name=collection_name_dense_sparse,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                limit=(10 * limit),
            ),
        ],
        query=models.Document(
            text=query,
            model="Qdrant/bm25", 
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

In [71]:
dense_sparse_search_eva = evaluate(ground_truth, lambda q: dense_sparse_search(q['question']))
print(dense_sparse_search_eva)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2715/2715 [00:35<00:00, 76.99it/s]

{'hit_rate': 0.8666666666666667, 'mrr': 0.7567280540208713}


In [76]:
results = dense_sparse_search("what is the best pub in Soho?")
for point in results:
    print(
        f"Score: {point.score}\n"
        f"City: {point.payload.get('city')}\n"
        f"Section: {point.payload.get('section')}\n"
        f"Subsection: {point.payload.get('subsection')}\n"
        f"Text: {point.payload.get('text')}"
    )
    print("=="*50)


Score: 12.055727
City: London
Section: Drink
Subsection: Pubs and bars
Text: Many local pubs, especially those run by chains like Wetherspoons and Scream, tend to be more reasonably priced with good drink promotions on weekday nights and during the day. As with the rest of the UK, chain pubs abound which Londoners tend to avoid like the plague. A good place to get cheap beer is at any one of the Sam Smith pubs found across Central London, including Soho and the City.
Score: 10.048122
City: London
Section: Stay safe
Subsection: Crime
Text: The main problem throughout London to various degrees is drunken behaviour, particularly on Friday and Saturday nights and after football matches. Loud and rowdy behaviour is to be expected and fights and acts of aggression also occur. If you are harassed, it is best to simply ignore and walk away from those concerned. Trouble spots can be expected around popular drinking locations such as Soho and in various suburban centres.
Score: 9.574356
City: Lo

# Hybrid Search

In [81]:
def rrf_search(query: str, limit: int = 5) -> list[models.ScoredPoint]:
    results = qdrant_client.query_points(
        collection_name=collection_name_dense_sparse,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                limit=(5 * limit),
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="Qdrant/bm25",
                ),
                using="bm25",
                limit=(5 * limit),
            ),
        ],
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True,
    )

    return results.points

In [82]:
hybrid_search_eva = evaluate(ground_truth, lambda q: rrf_search(q['question']))
print(hybrid_search_eva)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2715/2715 [01:42<00:00, 26.61it/s]

{'hit_rate': 0.9152854511970534, 'mrr': 0.8050549562980494}


In [84]:
results = rrf_search("Plan three days trip to Cairo?")
for point in results:
    print(
        f"Score: {point.score}\n"
        f"City: {point.payload.get('city')}\n"
        f"Section: {point.payload.get('section')}\n"
        f"Subsection: {point.payload.get('subsection')}\n"
        f"Text: {point.payload.get('text')}"
    )
    print("=="*50)


Score: 1.0
City: Cairo
Section: See
Subsection: Further afield
Text: Also, the pyramids of Saqqara and Memphis offer an easy day trip out of Cairo.
Score: 0.33333334
City: Cairo
Section: See
Subsection: None
Text: Pharaonic Village . Welcome to Egyptland! Especially if you've kids in tow, this showground and retail park will give you an instant Egypt-in-miniature. In Giza about 20 minutes drive from Downtown. ( updated Jan 2018 )
Score: 0.33333334
City: London
Section: See
Subsection: None
Text: Feeling overwhelmed? Not sure how to begin planning your trip? Transport for London have produced their own useful list of itineraries for tourists . They include famous landmarks and iconic transport icons on their various public transport networks, among others.
Score: 0.25396827
City: Cairo
Section: description
Subsection: None
Text: On the Nile river, Cairo is famous for its own history, preserved in the fabulous medieval Islamic city and Coptic sites in Old Cairo — with historic Cairo insc

# Evaluation Summary 

In [85]:
from tabulate import tabulate

In [141]:
# build table rows
table = [
    ["Semantic Search", "Semantic Search basline", basline_qd_search['hit_rate'], basline_qd_search['mrr']],
    ["Semantic Search", "Semantic Search with filter", filter_qd_search['hit_rate'], filter_qd_search['mrr']],
    ["Sparse vector", "Sparse vector with BM25", bm_qd_search['hit_rate'], bm_qd_search['mrr']],
    ["Multi-step search", "Dense and sparse", dense_sparse_search_eva['hit_rate'], dense_sparse_search_eva['mrr']],
    ["Hybrid Search", "Hybrid Search", hybrid_search_eva['hit_rate'], hybrid_search_eva['mrr']],
]

# print table
print(tabulate(table, headers=["Section", "Method", "Hit Rate", "MRR"], tablefmt="rounded_grid"))

╭───────────────────┬─────────────────────────────┬────────────┬──────────╮
│ Section           │ Method                      │   Hit Rate │      MRR │
├───────────────────┼─────────────────────────────┼────────────┼──────────┤
│ Semantic Search   │ Semantic Search basline     │   0.8593   │ 0.751283 │
├───────────────────┼─────────────────────────────┼────────────┼──────────┤
│ Semantic Search   │ Semantic Search with filter │   0.867403 │ 0.765513 │
├───────────────────┼─────────────────────────────┼────────────┼──────────┤
│ Sparse vector     │ Sparse vector with BM25     │   0.847145 │ 0.738686 │
├───────────────────┼─────────────────────────────┼────────────┼──────────┤
│ Multi-step search │ Dense and sparse            │   0.866667 │ 0.756728 │
├───────────────────┼─────────────────────────────┼────────────┼──────────┤
│ Hybrid Search     │ Hybrid Search               │   0.915285 │ 0.805055 │
╰───────────────────┴─────────────────────────────┴────────────┴──────────╯


# RAG Flow

In [142]:
from mistralai import Mistral
from mistralai.models import UserMessage
import os
from dotenv import load_dotenv


In [143]:
# loads variables from .env
load_dotenv()  

True

In [144]:
api_key = os.getenv("API_KEY")

In [145]:
llm_client = Mistral(api_key = api_key)

In [177]:
def build_prompt(query, search_results):
    context_template = "Q: {question}\n A: {text}"

    context_parts = []
    for point in search_results: 
        payload = point.payload
        context_parts.append(
            context_template.format(
                question=query,
                text=payload.get("text", "")
            )
        )

    context = "\n\n".join(context_parts)
    
    prompt_template = """
You're a travel assistant. Answer the QUESTION based on the CONTEXT from the traveller database.
Use only the facts from the CONTEXT when answering the QUESTION. If the QUESTION doesn't have a CONTEXT data just mention
you cannot answer the question.

QUESTION: {question}

CONTEXT:
{context}

    """.strip()
    
    prompt = prompt_template.format(question=query, context=context)
    return prompt


In [178]:
def llm(prompt):
    response = llm_client.chat.complete(
        model= "ministral-8b-latest",
        messages=[UserMessage(content=prompt)],
    )

    return response.choices[0].message.content

In [179]:
def rag(query):
    search_results = rrf_search(query=query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [180]:
answer = rag("Plan three trip to Cairo?")
print(answer)

Based on the provided context, here are three trip plans to Cairo:

1. **Historical Cairo and Pyramids:**
   - Day 1: Visit the Egyptian Museum, Khan al-Khalili bazaar, and the medieval Islamic city.
   - Day 2: Explore the Giza Pyramids and the nearby Saqqara Pyramid Complex.
   - Day 3: Relax at Al-Azhar Park and enjoy a felucca ride on the Nile.

2. **Modern Cairo and Suburbs:**
   - Day 1: Visit the Midan Tahrir area, Zamalek, and Ma'adi.
   - Day 2: Explore the Eastern districts (Heliopolis, Nasr City, and New Cairo).
   - Day 3: Visit the Cairo Citadel and enjoy the city's vibrant nightlife.

3. **Cultural and Foodie Cairo:**
   - Day 1: Visit the Coptic sites in Old Cairo and explore the Khan al-Khalili bazaar.
   - Day 2: Try local Egyptian cuisine at various restaurants and cafes.
   - Day 3: Visit the Cairo Citadel, enjoy a felucca ride on the Nile, and explore the city's fast food scene.


In [181]:
answer = rag("Plan three-day itinerary for Brussels?")
print(answer)

I cannot answer the question as there is no context data related to Brussels.
